In [39]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import login
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
HF_API_TOKEN = os.environ["HF_API_TOKEN"]

In [40]:
hf_embeddings = HuggingFaceEndpointEmbeddings(
    model="intfloat/multilingual-e5-large-instruct",
    # task = 'feature-extraction',
    huggingfacehub_api_token=HF_API_TOKEN
)

In [41]:
db = FAISS.load_local('db/KTAS_adult_faiss', hf_embeddings, allow_dangerous_deserialization=True)

In [42]:
def get_considerations(search_result):
    metadata = search_result[0][0].metadata
    primary = metadata['primary']
    secondary = metadata['secondary']

    return primary, secondary

In [43]:
llm = ChatOllama(model = "llama3.1:8b-instruct-q4_K_M")

In [44]:
output_parser = StrOutputParser()

In [45]:
with open ('../data/KTAS_symptoms.txt', 'r', encoding='utf-8-sig') as f:
    possible_symptoms = f.read()

possible_symptoms

'20주 미만의 임신\n20주 이상의 임신\n감각상실 / 이상감각\n감염 가능성 확인\n감염성 질환에 노출\n객혈\n경증 및 비특이적 증상 호소\n고혈당\n고혈압\n고환 통증 / 부종\n과다 복용\n과다호흡증후군\n관절 부종\n구토 / 구역\n국소성 부종 / 발적\n귀의 삼출물\n귀의 손상\n귀의 이물질\n기괴한 행동\n기억상실\n기침 / 코막힘\n기타 피부 상태\n눈 충혈 / 분비물\n눈 통증\n눈부심\n눈의 외상\n눈의 이물질\n눈의 화학물질 노출\n다뇨증\n다리 부기 / 부종\n단독 복부 외상 - 관통상\n단독 복부 외상 - 둔상\n단독 흉부 외상 - 관통상\n단독 흉부 외상 - 둔상\n동상 / 한랭손상\n두부 손상\n두통\n드레싱 교체\n딸꾹질\n떨림\n맥박이 없거나 차가운 사지\n목, 등, 허리 통증\n목의 부종 / 통증\n목의 외상\n몸통을 포함한 다발성 외상 - 관통상\n몸통을 포함한 다발성 외상 - 둔상\n물림\n물질 금단\n물질 오용 / 중독\n반지 제거\n발작\n발진\n변비\n보행 장애 / 운동 실조 / 강직\n복뷔 종괴 / 팽만\n복시\n복통\n불면증\n불안 / 위기 상황\n비정상 검사 결과\n사지 약화 / 뇌졸중 증상\n사회문제\n상기도감염 증상 호소\n상지 손상\n상지 통증\n상처 확인\n샅고랑 부위 통증  종괴\n생식기의 분비물 / 피부병변\n생식기의 외상\n석고 붕대 확인\n설사\n성폭행\n소변배출장애\n소양증\n수술 후 합병증\n숨참\n스테이플 / 봉합사 제거\n시력장애\n식욕부진\n실신 / 전실신\n심계항진 / 불규칙한 심장박동\n심정지(비외상성)\n심정지(외상성)\n쏘임\n안면 통증 (비외상성 / 비치아성)\n안명 외상\n안와주위 부종\n알레르기 반응\n연하장애 / 연하곤란\n열\n열상 / 천공\n영상 검사 / 검사실 검사\n옆구리 통증\n온열손상\n외상성 목, 등, 허리 손상\n외상없이 저절로 멍듦\n요로감염 증상\n우울증 / 자살 / 자해\n월경 문제\n유방의 발적 / 압통\n유해물질 흡입\n음경 부종\n음순 부종\n의료 장

### 테스트 1: LLM으로 바로 주증상 선택

In [ ]:
template = """You are a specialized medical symptom classification assistant. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must choose exactly one symptom from a predefined list of <Possible Symptoms>. 

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <Possible Symptoms> (including spacing, punctuation, letter case, etc.).
- Output only the final result without adding any additional text, explanations, or formatting.

Read the following information carefully and determine which single symptom from the <Possible Symptoms> best matches the information given.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}

<Possible Symptoms>
{possible_symptoms}"""

In [22]:
############ LLM 코드 작성 ##############

In [35]:
symptom = '코의 외상'

In [39]:
search_result = db.similarity_search_with_relevance_scores(symptom, k=1)

c:\Users\oneas\ONEASH_local\LLM-KTAS\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
primary, secondary = get_considerations(search_result)

### 테스트2: 주호소와 비고를 그대로 사용하여 vector DB에서 semantic search (top 1)

In [81]:
test_text = """주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함."""

In [94]:
search_result = db._similarity_search_with_relevance_scores(test_text, k=1)
primary, secondary = get_considerations(search_result)

c:\Users\oneas\ONEASH_local\LLM-KTAS\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [96]:
search_result[0][0].page_content

'사지 약화 / 뇌졸중 증상'

In [97]:
primary, secondary

([{'consideration': '중증 호흡곤란', 'KTAS_level': 1},
  {'consideration': '쇼크', 'KTAS_level': 1},
  {'consideration': '무의식', 'KTAS_level': 1},
  {'consideration': '중등도 호흡곤란', 'KTAS_level': 2},
  {'consideration': '혈역학적 장애', 'KTAS_level': 2},
  {'consideration': '의식변화', 'KTAS_level': 2},
  {'consideration': '열, 면역저하 상태', 'KTAS_level': 2},
  {'consideration': '패혈증 의증', 'KTAS_level': 2},
  {'consideration': '급성 중심성 중증 통증', 'KTAS_level': 2},
  {'consideration': '출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', 'KTAS_level': 2},
  {'consideration': '경증 호흡곤란', 'KTAS_level': 3},
  {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3},
  {'consideration': '전신염증반응증후군', 'KTAS_level': 3},
  {'consideration': '열 (아파 보임)', 'KTAS_level': 3},
  {'consideration': '급성 중심성 중등도 통증', 'KTAS_level': 3},
  {'consideration': '출혈성 질환 (중등도나 경도의 출혈)', 'KTAS_level': 3}],
 [{'consideration': '발병시간 24시간 이내', 'KTAS_level': 2},
  {'consideration': '발병시간 24시간 초과', 'KTAS_level': 3},
  {'consideration': '회복된 상태', 'KTAS_level': 3}])

### 테스트3: 주호소와 비고를 그대로 사용하여 vector DB에서 semantic search (top k) -> LLM이 가장 적절한 1개 선택

In [99]:
test_text = """주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함."""

In [100]:
search_result = db._similarity_search_with_relevance_scores(test_text, k=5)

c:\Users\oneas\ONEASH_local\LLM-KTAS\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


In [104]:
for doc in search_result:
    print(doc[0])

page_content='사지 약화 / 뇌졸중 증상' metadata={'ROS': '신경계', 'primary': [{'consideration': '중증 호흡곤란', 'KTAS_level': 1}, {'consideration': '쇼크', 'KTAS_level': 1}, {'consideration': '무의식', 'KTAS_level': 1}, {'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 2}, {'consideration': '패혈증 의증', 'KTAS_level': 2}, {'consideration': '급성 중심성 중증 통증', 'KTAS_level': 2}, {'consideration': '출혈성 질환 (생명 혹은 사지를 소실할 정도의 위급한 출혈)', 'KTAS_level': 2}, {'consideration': '경증 호흡곤란', 'KTAS_level': 3}, {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3}, {'consideration': '전신염증반응증후군', 'KTAS_level': 3}, {'consideration': '열 (아파 보임)', 'KTAS_level': 3}, {'consideration': '급성 중심성 중등도 통증', 'KTAS_level': 3}, {'consideration': '출혈성 질환 (중등도나 경도의 출혈)', 'KTAS_level': 3}], 'secondary': [{'consideration': '발병시간 24시간 이내', 'KTAS_level': 2}, {'consideration': '발병시간 24시간 초과', 'KTAS_level': 3}, {'cons

In [105]:
test_text = """주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함."""

In [109]:
search_result = db._similarity_search_with_relevance_scores(test_text, k=5)
for doc in search_result:
    print(doc[0])

page_content='기침 / 코막힘' metadata={'ROS': '호흡기계', 'primary': [{'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 2}, {'consideration': '패혈증 의증', 'KTAS_level': 2}, {'consideration': '경증 호흡곤란', 'KTAS_level': 3}, {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3}, {'consideration': '전신염증반응증후군', 'KTAS_level': 3}, {'consideration': '열 (아파 보임)', 'KTAS_level': 3}, {'consideration': '열 (건강해 보임)', 'KTAS_level': 4}], 'secondary': [{'consideration': '만성 기침 / 코막힘, 정상 활력징후', 'KTAS_level': 5}]}
page_content='저체온증' metadata={'ROS': '환경손상', 'primary': [{'consideration': '중증 호흡곤란', 'KTAS_level': 1}, {'consideration': '쇼크', 'KTAS_level': 1}, {'consideration': '무의식', 'KTAS_level': 1}, {'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 

In [111]:
db.similarity_search('fever and chilling')
for doc in search_result:
    print(doc[0])

page_content='기침 / 코막힘' metadata={'ROS': '호흡기계', 'primary': [{'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 2}, {'consideration': '패혈증 의증', 'KTAS_level': 2}, {'consideration': '경증 호흡곤란', 'KTAS_level': 3}, {'consideration': '비정상 맥박수지만 혈역학적으로 안정', 'KTAS_level': 3}, {'consideration': '전신염증반응증후군', 'KTAS_level': 3}, {'consideration': '열 (아파 보임)', 'KTAS_level': 3}, {'consideration': '열 (건강해 보임)', 'KTAS_level': 4}], 'secondary': [{'consideration': '만성 기침 / 코막힘, 정상 활력징후', 'KTAS_level': 5}]}
page_content='저체온증' metadata={'ROS': '환경손상', 'primary': [{'consideration': '중증 호흡곤란', 'KTAS_level': 1}, {'consideration': '쇼크', 'KTAS_level': 1}, {'consideration': '무의식', 'KTAS_level': 1}, {'consideration': '중등도 호흡곤란', 'KTAS_level': 2}, {'consideration': '혈역학적 장애', 'KTAS_level': 2}, {'consideration': '의식변화', 'KTAS_level': 2}, {'consideration': '열, 면역저하 상태', 'KTAS_level': 

### 테스트4: LLM으로 주호소 추출 (NER) → 주호소 참고하여 LLM으로 증상 선택 → 해당 증상으로 JSON 또는 vector DB에서 추출

In [ ]:
template_NER = """You are a specialized assistance for recognizing medical entities. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must extract all entities that correspond to the chief complaint or primary symptoms.  

Important Constraints:
- If multiple entities are present, output the in a single line separated by commas.
- Make sure to retain the exact text as it appears in the original note without any modifications.
- Output only the final result without adding any additional text, explanations, or formatting.

Read the following information carefully and extract all entities corresponding to the chief complaint or primary symptoms.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}"""

In [ ]:
prompt_NER = PromptTemplate.from_template(template_NER)

chain_NER = prompt_NER | llm | output_parser

In [ ]:
entities = chain_NER.invoke({"chief_complaint": "Disorientated", "additional_notes": "flapping tremor(-) 2차병원 내원하여 검사 안내함."})

In [33]:
entities

'Disorientated, flapping tremor'

In [36]:
template_symptom = """You are a specialized medical symptom classification assistant. You will receive a patient’s <Chief Coplaint> and you must choose exactly one symptom from a predefined list of <Possible Symptoms>. 

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <Possible Symptoms> (including spacing, punctuation, letter case, etc.).
- Output only the final result without adding any additional text, explanations, or formatting.

Read the following information carefully and determine which single symptom from the <Possible Symptoms> best matches the information given.

<Chief Complaint>:
{chief_complaint}

<Possible Symptoms>
{possible_symptoms}"""

In [46]:
prompt_symptom = PromptTemplate.from_template(template_symptom)

chain_symptom = prompt_symptom | llm | output_parser

In [47]:
symptom = chain_symptom.invoke({"chief_complaint": entities, "possible_symptoms": possible_symptoms})

In [48]:
symptom

'떨림'